# Die Class 

In [158]:
import numpy as np
import pandas as pd
import random



class Die():
    """This class, Die, creates an n dimensional die with equal, defualt weights, 
    or better thought of as probabilities, that can be adjusted as needed. 
    Additionally, this die can be rolled n many times and the state of the die, 
    also known as the compiled list of the n rolls can be retrieved at any point."""

    def __init__(self, faces):
        '''
        PURPOSE: initializes the weights from the input face values.

        INPUT: a NumPy array of either strings or integers that will name the sides of the die
        OUTPUT: a private dataframe with faces in the index.
        '''
        self.faces = faces
        if isinstance(self.faces,np.ndarray) != True:
            raise TypeError('faces argument is not a NumPy array. Please enter a NumPy array')
        if len(np.unique(faces)) != len(faces):
              raise ValueError('Please enter unique face names.')
        for n in self.faces:
            if type(n) != np.str_ and type(n) != np.int_ and type(n) != np.float_:
                raise TypeError('faces data type is not a string or integer. Please only enter strings or integers.')
        self.weights = np.ones(len(self.faces))
        self._facesweights = pd.DataFrame(self.weights, index = self.faces)

    def set_weight(self,facevalue,newweight):
        '''PURPOSE: to change the weight of the faces for the die.

            INPUT: the face value that you want to change, which should be a string or integer, 
            and the new weight you want to assign it which should be an integer or float
            OUTPUT: an updated dataframe that has the new weight values for the faces you wanted to change
            '''
        if facevalue not in self.faces:
            raise IndexError('The face value is not in the array of faces you input originally.')
        if (type(newweight) != int and type(newweight) != float):
            try:
                newweight = float(newweight)
            except TypeError:
                raise TypeError('The new weight is not an integer or float. Please input a float or integer for the new weight.')
        try:
            self._facesweights.loc[facevalue] = newweight
            self.weights = self._facesweights['Weights']
        except: KeyError

    def roll_dice(self,rolls = 1):
        '''PURPOSE: to randomly sample the die, aka roll the die, while taking into consideration 
        the number of faces the die has and their respective weights.

            INPUT: number of rolls, otherwise defaults to 1 roll
            OUTPUT: it adds the selection to a list and returns that list.
            '''
        roll_list = []
        for n in range(rolls):
            sample = random.choices(self.faces,weights=self.weights)[0]
            self.roll_list = roll_list.append(sample)
        return roll_list.copy()

    def get_dies_state(self):
        '''Method docstrings should describe the purpose of the 
        method, any input arguments, any return values if applicable, 
        and any changes to the object’s state that the user should 
        know about.'''
        return self._facesweights


In [294]:
test1 = np.array([1,2,3,4])
die1 = Die(test1)

test2 = np.array([1,2,3,4])
die2 = Die(test2)
die2.set_weight(2,2)
print(die2.get_dies_state())
die2.roll_dice(rolls = 10)


     0
1  1.0
2  2.0
3  1.0
4  1.0


[1, 1, 1, 4, 2, 2, 4, 3, 3, 1]

# Game Class

In [295]:
class Game():
    """This class, Game, rolls a list of instantiated die and only keep the results 
    of their most recent play. Each die has the same number of sides, and each game
    is initialized with a python list of one or more die.""" 
    def __init__(self, listofinstantiateddice):
        '''PURPOSE: 

            INPUT: 
            OUTPUT: 
            '''
        self.listofinstantiateddice = listofinstantiateddice
        self._private_outcome_df = pd.DataFrame()
    
    def play_game(self,numberoftimes = 1): 
        '''PURPOSE: 
         
            INPUT: 
            OUTPUT: 
        '''
        for x in self.listofinstantiateddice:
            self._private_outcome_df[f' die number {self.listofinstantiateddice.index(x)+1}'] = x.roll_dice(numberoftimes)
        self._private_outcome_df.index.name = "roll number"
        

    def most_recent(self, form = 'wide'):
        '''PURPOSE: 

            INPUT: 
            OUTPUT: 
            '''
        if form.lower() == 'narrow':
            return self._private_outcome_df.stack().copy()
        if form.lower() == 'wide':
            return self._private_outcome_df.copy()
        else:
            raise ValueError("Invalid option for dataframe form. Must be 'narrow' or 'wide'")



In [296]:
test3 = Game([die1,die2])
test3.play_game(numberoftimes=9)
test3.most_recent(form = 'wide')

,die number 1,die number 2
roll number,,
0,2,3
1,4,2
2,1,3
3,2,2
4,2,4
5,4,4
6,4,1
7,3,1
8,2,2


In [486]:
import numpy as np
import itertools

class Analyzer():
    """docstring Class docstrings should describe 
    the general purpose of the class."""
    def __init__(self,gameobject):
        '''PURPOSE: 

            INPUT: 
            OUTPUT: 
            '''
        if not isinstance(gameobject, Game):
            raise ValueError('This game object is not an instance of the Game class.')
        self.gameobject = gameobject

    def jackpot(self):
        '''PURPOSE: 

        INPUT: 
        OUTPUT: 
        '''
        self.counter = 0
        for i in self.gameobject._private_outcome_df.index:
            i = i-1
            row = np.array(self.gameobject._private_outcome_df.iloc[i],dtype=float)
            if len(np.unique(row)) == 1:
                self.counter += 1
        return self.counter
        

    def facecounts_per_roll(self):
        '''PURPOSE: 

        INPUT: 
        OUTPUT: 
        '''
        return pd.DataFrame(self.gameobject.most_recent()).apply(pd.Series.value_counts,axis = 1).fillna(0)


    def combo_count(self):
        '''PURPOSE: 

        INPUT: 
        OUTPUT: 
        '''
        combinationlist = list(itertools.combinations_with_replacement(
            list(self.gameobject.listofinstantiateddice[0].faces),
            len(self.gameobject.listofinstantiateddice)))
        combinationindex = pd.MultiIndex.from_tuples(combinationlist)
        combinationdf.index = [f'combination {i+1}' for i in range(len(self.gameobject.listofinstantiateddice))]
        combinationdf = pd.DataFrame(index=combinationindex)
        combinationdf['frequency'] = self.gameobject.most_recent().apply(lambda row: tuple(np.sort(row))).value_counts().to_frame()
        return combinationdf.fillna(0)


    def permutation_count(self):
        '''PURPOSE: 

        INPUT: 
        OUTPUT: 
        '''
        combinationlist = list(itertools.combinations_with_replacement(
            list(self.gameobject.listofinstantiateddice[0].faces),
            len(self.gameobject.listofinstantiateddice)))
        combinationindex = pd.MultiIndex.from_tuples(combinationlist)
        combinationdf.index = [f'combination {i+1}' for i in len(self.gameobject.listofinstantiateddice)]
        combinationdf = pd.DataFrame(index=combinationindex)
        combinationdf['frequency'] = self.gameobject.most_recent().apply(lambda row: tuple(np.sort(row))).value_counts().to_frame()
        return combinationdf.fillna(0)

In [488]:
test4 = Analyzer(test3)
test4.jackpot()
test4.facecounts_per_roll()
test4.combo_count()



UnboundLocalError: cannot access local variable 'combinationdf' where it is not associated with a value